In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pathlib import Path
from urllib.parse import urljoin, urlparse
import requests
import json
import time

In [2]:
BASE_URL = "https://manga.in.ua"
DOWNLOAD_FOLDER = Path("data")
MANGAS_PATH = Path("mangas.json")

with MANGAS_PATH.open(encoding="utf-8") as f:
    MANGAS = json.load(f)

In [3]:
# --- Налаштування Selenium ---
chrome_options = Options()
# chrome_options.add_argument("--headless")  # без GUI
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--log-level=3")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(5)

In [4]:
def download_chapter(driver: webdriver.Chrome, out_dir: Path):
    # кнопка "Читати розділ" (без except)
    read_btns = driver.find_elements(
        By.CSS_SELECTOR, 'a[onclick*="LoadMangaImages"]'
    )
    if read_btns:
        read_btns[0].click()
        print("Кнопка 'Читати розділ' натиснута")
    else:
        print("Кнопки 'Читати розділ' немає — контент вже завантажений")

    # чекаємо контейнер з картинками
    comics_div = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "comics"))
    )
    images = WebDriverWait(comics_div, 20).until(
        lambda d: d.find_elements(By.CSS_SELECTOR, "img[data-src]")
    )
    if not images:
        raise RuntimeError("Не знайдено картинок у розділі")

    # завантажуємо
    out_dir.mkdir(parents=True, exist_ok=True)
    for idx, img in enumerate(images, start=1):
        img_url = urljoin(BASE_URL, img.get_attribute("data-src"))
        ext = Path(urlparse(img_url).path).suffix or ".jpg"
        filename = out_dir / f"{idx:04d}{ext}"
        filename.write_bytes(requests.get(img_url).content)

    print(f"Завантажено {len(images)} сторінок у {out_dir}")

In [5]:
for manga_name, start_url in MANGAS.items():
    print(f"\n===== МАНГА: {manga_name} =====")

    driver.get(start_url)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "comics"))
    )

    chapter_i = 1
    while True:
        chapter_url = driver.current_url
        folder_name = chapter_url.split("/")[-1].replace(".html", "")
        folder_name = f"{chapter_i:04d}_{folder_name}"
        chapter_i += 1
        out_folder = DOWNLOAD_FOLDER / manga_name / folder_name

        print(f"\n=== Розділ: {folder_name} ===")

        try:
            download_chapter(driver, out_folder)
        except Exception as e:
            print(f"Помилка розділу: {e}")
            break

        # кнопка "Наступний"
        next_btns = driver.find_elements(By.ID, "fwdTop")
        if not next_btns:
            print("Кнопка 'Наступний' відсутня — кінець манги")
            break

        next_btn = next_btns[0]
        if "disabled" in next_btn.get_attribute("class"):
            print("Останній розділ манги")
            break

        next_btn.click()
        time.sleep(5)

    print(f"Манга '{manga_name}' завершена")


===== МАНГА: Токійський гуль =====

=== Розділ: 0001_28492-tokijskij-gul-tom-1-rozdil-1 ===
Кнопка 'Читати розділ' натиснута
Завантажено 47 сторінок у data/Токійський гуль/0001_28492-tokijskij-gul-tom-1-rozdil-1

=== Розділ: 0002_28890-tokijskij-gul-tom-1-rozdil-2 ===
Кнопка 'Читати розділ' натиснута
Завантажено 30 сторінок у data/Токійський гуль/0002_28890-tokijskij-gul-tom-1-rozdil-2

=== Розділ: 0003_29352-tokijskij-gul-tom-1-rozdil-3 ===
Кнопка 'Читати розділ' натиснута
Завантажено 26 сторінок у data/Токійський гуль/0003_29352-tokijskij-gul-tom-1-rozdil-3

=== Розділ: 0004_29830-tokijskij-gul-tom-1-rozdil-4 ===
Кнопка 'Читати розділ' натиснута
Завантажено 19 сторінок у data/Токійський гуль/0004_29830-tokijskij-gul-tom-1-rozdil-4

=== Розділ: 0005_30205-tokijskij-gul-tom-1-rozdil-5 ===
Кнопка 'Читати розділ' натиснута
Завантажено 18 сторінок у data/Токійський гуль/0005_30205-tokijskij-gul-tom-1-rozdil-5

=== Розділ: 0006_30664-tokijskij-gul-tom-1-rozdil-6 ===
Кнопка 'Читати розділ'